# Regression in Marketing A/B Testing

Simple marketing campaign with experiment and control group for A/B testing [(Kaggle dataset)](https://www.kaggle.com/datasets/faviovaz/marketing-ab-testing/data).

The majority of the people will be exposed to ads (the experimental group). And a small portion of people (the control group) would instead see a Public Service Announcement (PSA) (or nothing) in the exact size and place the ad would normally be.

**Data dictionary:**

- user id: User ID (unique)
- test group: If "ad" the person saw the advertisement, if "psa" they only saw the public service announcement
- converted: If a person bought the product then True, else is False
- total ads: Amount of ads seen by person
- most ads day: Day that the person saw the biggest amount of ads
- most ads hour: Hour of day that the person saw the biggest amount of ads

In [1]:
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

In [2]:
# Read data and clean up column names
df = pd.read_csv('marketing_campaign.csv', index_col=0)
df = df.rename(columns=lambda x: x.strip().replace(" ", "_"))
df['treatment'] = (df['test_group'] == 'ad').astype(int)
df['converted'] = df['converted'].astype(int)
df.head()

,user_id,test_group,converted,total_ads,most_ads_day,most_ads_hour,treatment
0,1069124,ad,0,130,Monday,20,1
1,1119715,ad,0,93,Tuesday,22,1
2,1144181,ad,0,21,Tuesday,18,1
3,1435133,ad,0,355,Tuesday,10,1
4,1015700,ad,0,276,Friday,14,1


## Regression for A/B Testing

$$
\text{converted} = \beta_0 + \beta_1 \times \text{treatment} + \epsilon
$$

In [3]:
model_base = ('converted ~ treatment')
base = smf.ols(model_base, data=df)
results_base = base.fit(cov_type='HC1')
results_base.summary().tables[1]

,coef,std err,z,P>|z|,[0.025,0.975]
Intercept,0.0179,0.001,20.679,0.000,0.016,0.020
treatment,0.0077,0.001,8.657,0.000,0.006,0.009


In [4]:
base_est = results_base.params['treatment']
base_se = results_base.bse['treatment']
base_ci = results_base.conf_int().loc['treatment']
print(f"The effect of the treatment is: {base_est*100:.2f}%")
print(f"Standard error: {base_se*100:.3f}%")
print(f"Confidence interval: {base_ci[0]*100:.2f}% - {base_ci[1]*100:.2f}%")

The effect of the treatment is: 0.77%
Standard error: 0.089%
Confidence interval: 0.60% - 0.94%


## Additive Regression Model

$$
\text{converted} = \beta_0 + \beta_1 \times \text{treatment} + \beta_2 \times \text{total ads} + \beta_3 \times \text{most ads day} + \beta_4 \times \text{most ads hour} + \epsilon
$$

In [5]:
model_add = ('converted ~ treatment + total_ads + most_ads_day + C(most_ads_hour)')
add = smf.ols(model_add, data=df)
results_add = add.fit(cov_type='HC1')
results_add.summary().tables[1]

,coef,std err,z,P>|z|,[0.025,0.975]
Intercept,-0.0112,0.002,-5.563,0.000,-0.015,-0.007
most_ads_day[T.Monday],0.0115,0.001,15.185,0.000,0.010,0.013
most_ads_day[T.Saturday],-0.0002,0.001,-0.298,0.766,-0.002,0.001
most_ads_day[T.Sunday],0.0040,0.001,5.684,0.000,0.003,0.005
most_ads_day[T.Thursday],0.0019,0.001,2.737,0.006,0.001,0.003
most_ads_day[T.Tuesday],0.0100,0.001,13.159,0.000,0.009,0.012
most_ads_day[T.Wednesday],0.0047,0.001,6.485,0.000,0.003,0.006
C(most_ads_hour)[T.1],-0.0070,0.002,-2.906,0.004,-0.012,-0.002
C(most_ads_hour)[T.2],-0.0125,0.002,-5.823,0.000,-0.017,-0.008
C(most_ads_hour)[T.3],-0.0075,0.003,-2.831,0.005,-0.013,-0.002


In [6]:
add_est = results_add.params['treatment']
add_se = results_add.bse['treatment']
add_ci = results_add.conf_int().loc['treatment']
print(f"The effect of the treatment is: {add_est*100:.2f}%")
print(f"Standard error: {add_se*100:.3f}%")
print(f"Confidence interval: {add_ci[0]*100:.2f}% - {add_ci[1]*100:.2f}%")

The effect of the treatment is: 0.76%
Standard error: 0.087%
Confidence interval: 0.59% - 0.93%


## Interactive Regression Model

$$
\text{converted} = \beta_0 + \beta_1 \times \text{treatment} + \beta_2 \times \text{treatment} \times \text{total ads} + \beta_3 \times \text{treatment} \times \text{most ads day} + \beta_4 \times \text{treatment} \times \text{most ads hour} + \epsilon
$$

In [7]:
# Demean covariates
formula = ('0 ~ total_ads + most_ads_day + C(most_ads_hour)')
X = patsy.dmatrix(formula, df, return_type='dataframe')
X = X.drop('Intercept', axis=1)
X.columns = [f'x{t}' for t in range(X.shape[1])] # clean column names
X = (X - X.mean(axis=0))

# Construct interactions
formula = "treatment * (" + "+".join(X.columns) + ")"
X['treatment'] = df['treatment']
X = patsy.dmatrix(formula, X, return_type='dataframe')
int = sm.OLS(df[['converted']], X)
results_int = int.fit(cov_type='HC1')
# results_int.summary().tables[1]

In [8]:
int_est = results_int.params['treatment']
int_se = results_int.bse['treatment']
int_ci = results_int.conf_int().loc['treatment']
print(f"The effect of the treatment is: {int_est*100:.2f}%")
print(f"Standard error: {int_se*100:.3f}%")
print(f"Confidence interval: {int_ci[0]*100:.2f}% - {int_ci[1]*100:.2f}%")

The effect of the treatment is: 0.78%
Standard error: 0.087%
Confidence interval: 0.61% - 0.95%


## Fully Interactive Regression Model

In [9]:
# Demean covariates
formula = ('0 ~ (total_ads + most_ads_day + C(most_ads_hour))**2')
X = patsy.dmatrix(formula, df, return_type='dataframe')
X = X.drop('Intercept', axis=1)
X.columns = [f'x{t}' for t in range(X.shape[1])] # clean column names
X = (X - X.mean(axis=0)) # demean all control covariates

# Construct interactions
formula = "treatment * ("+ "+".join(X.columns) + ")"
X['treatment'] = df['treatment']
X = patsy.dmatrix(formula, X, return_type='dataframe')
full = sm.OLS(df[['converted']], X)
results_full = full.fit(cov_type='HC1')
# results_full.summary().tables[1]

In [10]:
full_est = results_full.params['treatment']
full_se = results_full.bse['treatment']
full_ci = results_full.conf_int().loc['treatment']
print(f"The effect of the treatment is: {full_est*100:.2f}%")
print(f"Standard error: {full_se*100:.3f}%")
print(f"Confidence interval: {full_ci[0]*100:.2f}% - {full_ci[1]*100:.2f}%")

The effect of the treatment is: 0.78%
Standard error: 0.088%
Confidence interval: 0.60% - 0.95%


In [11]:
print(f"There are {X.shape[1]} covariates in the model.")

There are 396 covariates in the model.
